In [1]:
import tensorflow as tf
from tensorflow import keras
from packaging import version
from datetime import datetime
import pandas as pd
import numpy as np
import os
import glob

In [2]:
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.11.0


In [3]:
import tensorboard
tensorboard.__version__

'2.11.0'

In [4]:
from linkmethods import get_pageviews

In [5]:
from keras.models import Sequential
from keras.layers import *

In [49]:
!rm -rf ./logs/

Der Befehl "rm" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [9]:
path= os.path.abspath(r"C:\Users\Marvin\PycharmProjects\pythonProject\datafiles")

syear = 2015
start = datetime(syear, 7, 1)
end = datetime(2022, 7, 1)
data = get_pageviews("Meisen", start, end, project="de.wikipedia.org")
df_meisen = pd.DataFrame(data)[["timestamp","views"]]
df_meisen["timestamp"] = pd.to_numeric(df_meisen["timestamp"])
df_meisen = df_meisen.set_index("timestamp")

df_meisen

,views
timestamp,
2015070100,220
2015070200,222
2015070300,218
2015070400,228
2015070500,287
...,...
2022062700,208
2022062800,192
2022062900,190


In [29]:
from zeitreihenanalyse import pageviewget
from linksAnalysis import get_back_links

In [12]:
abs_path = r"C:\Users\Marvin\PycharmProjects\pythonProject\datafiles\Meisen\Blinks"
path = os.path.abspath(abs_path)
csv_files = glob.glob(os.path.join(path, "*.csv"))

flatten_views = []

for f in csv_files:
    df = pd.read_csv(f)
    df = df[["timestamp","views"]].set_index("timestamp").reindex_like(df_meisen).fillna(0)

    views = df.views.to_numpy()
    flatten_views.extend(views)

In [35]:
data = np.array(flatten_views).reshape((36,2558,1))
data.shape

(36, 2558, 1)

In [33]:
target = np.array(df_meisen.views)
target = np.array([np.copy(target) for _ in range((data.shape[0]))])

target.shape

(36, 2558)

In [152]:
# Define the model.
model = Sequential()

# 1 dense hidden layer with 250 nodes ~90+ acc
model.add(Input(shape=(2558,))) # Input tensor
model.add(Dense(units=250)) # hidden layer 1,
model.add(Dense(units=2558)) #output layer


model.compile(
    optimizer='adam',
    loss='MeanAbsolutePercentageError',
    metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model.
model.fit(
    data,
    target,
    batch_size=5,
    epochs=40,
    callbacks=[tensorboard_callback])

Epoch 1/40
8/8 [==============================] - 0s 22ms/step - loss: 118.5932 - accuracy: 0.0000e+00
Epoch 2/40
8/8 [==============================] - 0s 24ms/step - loss: 114.5858 - accuracy: 0.0000e+00
Epoch 3/40
8/8 [==============================] - 0s 23ms/step - loss: 93.7171 - accuracy: 0.0000e+00
Epoch 4/40
8/8 [==============================] - 0s 21ms/step - loss: 87.8072 - accuracy: 0.0000e+00
Epoch 5/40
8/8 [==============================] - 0s 22ms/step - loss: 90.4610 - accuracy: 0.1111
Epoch 6/40
8/8 [==============================] - 0s 21ms/step - loss: 94.6742 - accuracy: 0.5833
Epoch 7/40
8/8 [==============================] - 0s 22ms/step - loss: 93.1081 - accuracy: 0.1944
Epoch 8/40
8/8 [==============================] - 0s 22ms/step - loss: 120.1042 - accuracy: 0.8333
Epoch 9/40
8/8 [==============================] - 0s 21ms/step - loss: 109.7237 - accuracy: 0.7500
Epoch 10/40
8/8 [==============================] - 0s 23ms/step - loss: 92.6820 - accuracy: 0.8611

In [124]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [70]:
%tensorboard --logdir logs/fit --host localhost --port 8088

Launching TensorBoard...

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (2022872164.py, line 1)